<a href="https://colab.research.google.com/github/neenumohan/LSTM/blob/main/LSTM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
#Download the dataset
path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#Explore the data
text = open(path, "r").read()
print(len(text))
print(text[:200])

1115394/1115394 [==============================] - 0s 0us/step
1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [ ]:
# one-hot encode
maxlen = 60 #extract sequences of length 60
step = 3
sentences = []	#holds extracted sequences
next_chars = [] #holds the targets
for i in range(0, len(text)-maxlen, step):
	sentences.append(text[i:i+maxlen])
	next_chars.append(text[i+maxlen])
#VECTORIZATION
chars = sorted(set(text))
char_indices = dict((char, chars.index(char)) for char in chars)
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.int)
y = np.zeros((len(sentences), len(chars)), dtype=np.int)
for i, sentence in enumerate(sentences):
	for t, char in enumerate(sentence):
		x[i, t, char_indices[char]] = 1
	y[i, char_indices[next_chars[i]]] = 1

<ipython-input-2-15e89c42c865>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.int)
<ipython-input-2-15e89c42c865>:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
D

In [ ]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam",run_eagerly=True)

In [ ]:
def sample(preds, temperature=1.0):
	preds = np.asarray(preds).astype('float64')
  #print("pred1:",preds)
	preds = np.log(preds) / temperature
	#print("pred2:",preds)
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
  #print("probas:",probas)
	return np.argmax(probas)

In [ ]:
# @title Default title text
#The following loop repeatedly trains and generates the text. #generates 400 length string
import random
for epoch in range(1, 60):
  model.fit(x, y, batch_size=128)
  start_index = random.randint(0, len(text) - maxlen - 1)
  print(start_index)
  generated_text = x[start_index: start_index + maxlen]
  for i in range(10):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(i)[0]
    print(preds)
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
  print("...Generated: ", generated_text)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               99328     
                                                                 
 dense (Dense)               (None, 65)                8385      
                                                                 
Total params: 107,713
Trainable params: 107,713
Non-trainable params: 0
_________________________________________________________________
